In [1]:
"""
1.准备数据(读取csv)
2.构建模型：adaboost+决策树
3.打印MSE
4.保存模型
"""

import numpy as np                                                       # 处理多维数组（矩阵）模块
import pandas as pd                                                      # 数据分析模块
# 1.准备数据(读取csv)
data = pd.read_csv('061.csv')
data = data.values                                        # 将表格转换为矩阵 print(data.as_matrix()) 相同效果


# 分割数据
from sklearn.model_selection import train_test_split                    # 分割数据集模块
                                                         # （用于将矩阵随机划分为训练子集和测试子集，并返回划分好的训练集测试集数据样本和训练集测试集标签）
trainSet,testSet = train_test_split(data,test_size=0.3)  # train 训练 test 测试 test_size 设置训练集分割比例
trainX,trainY = trainSet[:,:-1],trainSet[:,-1]           # trainX 0.7 trainY 0.7   
testX,testY = testSet[:,:-1],testSet[:,-1]               # testX 0.3 textY 0.3
                                                         #（拆分出训练集的X,y；测试集的X,y，取所有的前两列，取所有的最后一列，X为数据 Y为分类）
    
# 建模
from sklearn.tree import DecisionTreeClassifier                         # 决策树的分类模型模块
from sklearn.ensemble import AdaBoostClassifier                         # 集成模块 的算法 AdaBoost(提升算法)Classifier分类器
                                                         #（使用集成算法：可以生成多颗决策树，通过将多个弱分类器组合成一个强分类器）
tree = DecisionTreeClassifier(max_depth=5)               # 获取一个分类器对象，为了把训练任务交给adaboost
adaBoostC = AdaBoostClassifier(tree,n_estimators=300)    # 参数：给出一个弱分类器，n_estimators 设置最大弱分类器的数量

# 训练数据
adaBoostC.fit(trainX,trainY)                             # 拿出训练数据0.7 训练分类0.7 训练 决策树分类模型（fit训练方法：可以理解为一个训练过程）

# 预测数据
y_pre_test = adaBoostC.predict(testX)                    # 使用训练后的模型 预测 测试数据0.3 得出对应的0.3分类(predict预测方法：返回预测分类)
print(y_pre_test)                                        # 预测的分类值0.3
print(testY)                                             # 真实的分类值0.3


# 数据集可视化（散点图）
from matplotlib import pyplot as plt                                    # 绘图库中的绘图命令模块

def visual_2D_dataset(dataset_X,dataset_y):               # 创建一个2D绘图函数，两个参数：二维数据集dataset_X和对应的类别dataset_y显示在散点图中
    assert dataset_X.shape[1]==2,'only support dataset with 2 features'  # 判断数据集是不是有两个特征 自定义错误：只支持具有两个特征的数据集
                                                          # assert 断言关键字 用于在代码中设置检查点,测试后面表示式，其返回值为假，就会触发异常中断程序执行。
    plt.figure()                                          # 初始化一张画板
    classes=list(set(dataset_y))                          # 把所有分类放在列表中 并去重 set()方法中的数据，无序，且内容不能重复
    markers=['.',',','o','v','^','<','>','1','2','3','4','8','s','p','*','h','H','+','x','D','d','|']  # 设置散点符号
    colors=['b','c','g','k','m','r','y']                  # 设置散点颜色
    for class_id in classes:                              # 循环列表中的所有分类
        one_class=np.array([feature for (feature,label) in zip(dataset_X,dataset_y) if label==class_id]) 
        # 生成高维数组 zip方法将多个list的数据 合并 成元祖的list  [(数据,分类),()]  如果label类属于其中一个类 取出label所对应的特征数据生成一个one_class特征数据
        plt.scatter(one_class[:,0],one_class[:,1],marker=np.random.choice(markers,1)[0],c=np.random.choice(colors,1)[0],label='class_'+str(class_id))
        # 绘制散点图方法 取出所有的one_class的第一列,第二列,并设置随机选取一个散点符,随机选取一个颜色,设置图例
        # np.random.choice(['1','2','3'],2)[1]  随机选取方法 参数为数据，选取几个 返回一个array类型 通过下标可以取出其中数据

    plt.legend()                                         # 绘制图例
    plt.show()                                           # 绘制图
visual_2D_dataset(testX,testY)                           # 函数传参真实值
visual_2D_dataset(testX,y_pre_test)                      # 函数传参预测值

# 评估模型
from sklearn.metrics import classification_report                       # 模型评估模块中的分类报告模块
report = classification_report(y_pred=y_pre_test,y_true=testY)  # 测试值与真实值对比计算出分类准确率
print(report)

# 可视化混淆矩阵
from sklearn.metrics import confusion_matrix                            # 模型评估模块中的混淆矩阵模块
conf_matrix = confusion_matrix(y_pred=y_pre_test,y_true=testY)  # 测试值与真实值对比计算出一个混淆矩阵数据
print(conf_matrix)

import itertools                                                        # 迭代器模块：用于操作迭代对象的函数集合
                     # 它是处理迭代功能的函数，它们的返回值不是list，而是迭代对象，只有用for循环迭代的时候才真正计算
from matplotlib import pyplot as plt                                    # 绘图库中的绘图命令模块
import numpy as np                                                      # 处理多维数组（矩阵）模块
import matplotlib                                                       # 绘图库模块

matplotlib.rcParams['font.sans-serif'] = ['SimHei']             # 设置字体为SimHei黑体 显示中文

def plot_confusion_matrix(confusion_mat):                       # 定义一个混淆矩阵图函数传入一个参数为混淆矩阵数据   将混淆矩阵画图并显示出来
    # 热图（heatmap）是数据分析的常用方法，通过色差、亮度来展示数据的差异、易于理解。Python在Matplotlib库中，调用imshow()函数实现热图绘制。
    plt.imshow(confusion_mat, interpolation='nearest', cmap=plt.cm.Blues)
    # plt.imshow()热图（斑图）绘制方法 参数：矩阵数据，interpolation 图片的模糊度 cmap 设置颜色 plt.cm中cm全称表示colormap(调色板)
    # interpolation的值由深到浅依次为：‘none’，‘nearest’，‘bilinear’，‘bicubic’，‘spline16’, ‘spline36’, ‘hanning’, 
    # ‘hamming’,‘hermite’，‘kaiser’，‘quadric’，‘catrom’，‘gaussian’，‘bessel’，‘mitchell’， ‘sinc’，‘lanczos’
    plt.title('混淆矩阵')                                       # plt.title方法 设置矩阵标题
    plt.colorbar()                                             # plt.colorbar方法 给子图添加colorbar（颜色条或渐变色条）
                                                               # 当我们给图配渐变色时，常常需要在图旁边把colorbar显示出来
    
    tick_marks = np.arange(confusion_mat.shape[0])             # 生成区间数组 取出混淆数据的子维度数(总类数)
    plt.xticks(tick_marks, tick_marks)                         # 设置x轴的刻度 参数为长度，刻度值
    plt.yticks(tick_marks, tick_marks)                         # 设置y轴的刻度
    thresh = confusion_mat.max() / 2.                          # 取出矩阵数据的最大值  
    # product(list1, list2) 方法依次取出list1中的每1个元素，与list2中的每1个元素，组成元组， 然后，将所有的元组组成一个列表，返回。
    for i, j in itertools.product(range(confusion_mat.shape[0]), range(confusion_mat.shape[1])):   # 取出子维度数，个数 分别赋值给i,j
        plt.text(j, i, confusion_mat[i, j],horizontalalignment="center",color="white" if confusion_mat[i, j] > thresh else "black")
        # plt.text方法 给图形添加数据标签 坐标值标签 horizontalalignment 平面线型居中 如果坐标的值大于 中间值 设置为白色 否则 设置为黑色
        
    plt.tight_layout()           # tight_layout会自动调整子图参数，使之填充整个图像区域。它仅仅检查坐标轴标签、刻度标签以及标题的部分 使其不重合。
    plt.ylabel('真实值')                                         # 配置x轴的标记
    plt.xlabel('预测值')                                         # 配置y轴的标记
    plt.show()                                                  # 绘制图
plot_confusion_matrix(conf_matrix)                              # 函数传参


[0. 1. 1. 3. 0. 2. 2. 3. 1. 0. 2. 2. 0. 1. 1. 2. 1. 3. 1. 1. 0. 0. 3. 3.
 3. 3. 3. 1. 3. 0. 2. 0. 1. 2. 0. 0. 0. 0. 3. 2. 1. 2. 0. 2. 0. 0. 2. 2.
 2. 1. 3. 3. 2. 0. 2. 2. 1. 3. 3. 1. 0. 3. 0. 3. 3. 3. 2. 3. 2. 1. 0. 3.
 1. 0. 3. 3. 1. 3. 1. 2. 1. 0. 1. 3. 2. 0. 1. 0. 0. 0. 0. 0. 0. 3. 2. 3.
 2. 0. 1. 1. 0. 0. 1. 0. 2. 1. 1. 1. 2. 2. 1. 0. 2. 1. 0. 0. 2. 2. 2. 1.]
[0. 1. 1. 3. 0. 2. 2. 3. 1. 0. 2. 2. 0. 1. 1. 2. 1. 3. 1. 1. 0. 0. 3. 3.
 3. 3. 3. 1. 3. 0. 2. 0. 1. 2. 0. 0. 0. 0. 3. 2. 1. 2. 0. 2. 0. 0. 2. 2.
 2. 1. 2. 3. 2. 0. 2. 2. 1. 3. 3. 1. 0. 3. 0. 3. 3. 3. 2. 3. 2. 1. 0. 3.
 1. 0. 3. 3. 1. 3. 1. 2. 1. 0. 1. 3. 2. 0. 1. 0. 0. 0. 0. 0. 0. 3. 2. 3.
 2. 0. 1. 1. 3. 0. 1. 0. 2. 1. 1. 1. 2. 2. 1. 0. 2. 1. 0. 0. 2. 2. 2. 1.]


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99        34
         1.0       1.00      1.00      1.00        30
         2.0       1.00      0.97      0.98        30
         3.0       0.96      0.96      0.96        26

    accuracy                           0.98       120
   macro avg       0.98      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120

[[34  0  0  0]
 [ 0 30  0  0]
 [ 0  0 29  1]
 [ 1  0  0 25]]


<Figure size 640x480 with 2 Axes>

In [25]:
np.random.choice(['1','2','3'],2)

array(['3', '1'], dtype='<U1')

In [16]:
conf_matrix[0,0]

33